In [ ]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup as BS 
import json

# defining the data frame where we have to store the Data from WEB
col = ["CET_no",'Name','Rank','Verfied_cat','Cat_alloted','College_alloted','course','SNAO','Fees']
df = pd.DataFrame(columns=col)

# loading  PGCET numbers from JSON file
with open('pgect_no.json','r') as  file:
    data = json.load(file)

# fetching all the PGCET Numbers
pgcet_numbers_list = data.get('pgect_no',[])


# fetchinng all the data from the data from website through url
start_number = 0 # this is just to start from that last number featched to csv file if something happens while the loop is running

for number in range(start_number, len(pgcet_numbers_list)):
    try:
        url = 'https://keaonline.karnataka.gov.in/pgcet_result_2024/main/resultscheck.php'  # Replace with your URL
        payload = {
            'txtrollno':pgcet_numbers_list[number]
        }
        response = requests.post(url, data=payload)
        res_text = response.text
        
        # convertint text to readable HTML using Beautifull soup
        HTML_text = BS(res_text,'html.parser')
        lst_1 = HTML_text.body.find_all('table')[2].find_all('table')[1].text.split("\n")
        lst = list(filter(lambda x: (x != '' and x != '\r'),lst_1))
        
        df.loc[number,col[0]] = lst[4].strip() # CET_no
        df.loc[number,col[1]] = lst[7].strip() # Name
        df.loc[number,col[2]] = int(lst[13].strip()[:-1].split('-')[-1].strip()) # Rank
        df.loc[number,col[3]] = lst[10].strip() # Verfied_cat
        df.loc[number,col[4]] = lst[24].strip() # Cat_alloted
        df.loc[number,col[5]] = lst[18].strip() # College_alloted
        df.loc[number,col[6]] = lst[15].strip() #course
        df.loc[number,col[7]] = int(lst[27].strip()) # SNAO(Serial Number of the Allotted Option)
        df.loc[number,col[8]] = lst[-1].strip() # Fees
        if number % 100 == 0:
            print("updated to CSV upto", number, pgcet_numbers_list[number])
            df.to_csv("MCA.csv")
    except:
        print("unable to fetch : ",pgcet_numbers_list[number])
df.to_csv("MCA.csv")
